In [ ]:
import requests 
from bs4 import BeautifulSoup
import datetime

In [ ]:
def get_movie_url(x=1980,y=2018): 
    if x > y :
        raise ValueError('Start time should be earlier.')
    if x < 1980 or y> int(str(datetime.datetime.now())[:4]):
        raise ValueError('Please input time during 1980 till now.')
    
    url = 'https://www.boxofficemojo.com/yearly/'
    url_list = []
    
    response = requests.get(url)
    results = BeautifulSoup(response.content, 'lxml')
    results = results.find_all('a')
    
    for result in results:
        if 'yr=' in result.get('href') and int(result.get_text())>=x and int(result.get_text())<=y:
            url_list.append(url+result.get('href'))
    return url_list

In [ ]:
def get_movie(url = None, year = None):
    if url == None:
        url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
    response = requests.get(url)
    results = BeautifulSoup(response.content, 'lxml')
    results = results.find_all('a')        
    
    url_list=[]
    for result in results:
        if result.get('href') != None:
            if 'id='in result.get('href') and '#' not in result.get_text():
                url_list.append('https://www.boxofficemojo.com'+result.get('href'))
    return url_list

In [ ]:
def get_movie_name(url = None, year = None):
    if url == None:
        url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
    response = requests.get(url)
    results = BeautifulSoup(response.content, 'lxml')
    results = results.find_all('a')        
    
    moviename_list=[]
    for result in results:
        if result.get('href') != None:
            if 'id='in result.get('href') and '#' not in result.get_text():
                moviename_list.append(result.get_text())
    return moviename_list

In [ ]:
# return a dict of (movie_name, movie_url)
def find_all(x,y):
    list_url = get_movie_url(x,y)
    dict_movie = {}
    for url in list_url:
        list_0 = dict_movie.copy()
        list_1 = dict(map(lambda x,y:[x,y],get_movie_name(url),get_movie(url)))
        dict_movie = {**list_0, **list_1}
    return dict_movie

In [ ]:
#  a specifi movie's daily gross
def gross_movie(name,begin_year=1980,end_year=2018):      
    import pandas as pd
    import requests 
    from bs4 import BeautifulSoup
    try:
        movie_url = find_all(begin_year,end_year)[name]
        id_position = movie_url.find('id=')
        name_id = movie_url[id_position:]
        url = 'https://www.boxofficemojo.com/movies/?page=daily&view=chart&'+name_id
        response = requests.get(url)
        results = BeautifulSoup(response.content, 'lxml')
        tables = results.select('table')
        df_list = []
        for table in tables:
            df_list.append(pd.concat(pd.read_html(table.prettify())))
        movie_gross = df_list[7]
        movie_gross.columns = ['Day','Date','Rank','Gross','%daily-change','%weekly-change','Theaters','Average','Gross-to-Date','Days']
        movie_gross = movie_gross[(movie_gross['Date'] != 'Date') & (movie_gross['Date'].notnull())]
        movie_gross.set_index('Date',inplace=True)
        movie_gross[['Days']] = movie_gross[['Days']].astype(int)
        return movie_gross        
    except:
        ValueError(f'{name} is not in the TOP100 of year from to {begin_year} to {end_year}')

In [ ]:
 # a specifi movie's basic information
def data_movie(name,begin_year=1980,end_year=2018):      
    import pandas as pd
    import requests 
    from bs4 import BeautifulSoup
    try:
        movie_url = find_all(begin_year,end_year)[name]
        response = requests.get(movie_url)
        results = BeautifulSoup(response.content, 'lxml')
        tables = results.select('table')
        df_list = []
        for table in tables:
            df_list.append(pd.concat(pd.read_html(table.prettify())))
        movie_info = {}
        movie_info['Name'] = name
        
        movie_info['Genre'] = df_list[5][0].iloc[2][8:]
        movie_info['Runtime'] = df_list[5][1].iloc[2][10:]
        movie_info['Production Budget'] = df_list[5][1].iloc[3][20:]
        
        movie_info['Domestic Gross'] = df_list[9][1].iloc[0]
        try:
            movie_info['Foreign Gross'] = df_list[9][1].iloc[1]
        except:
            movie_info['Foreign Gross'] = None
        try:
            movie_info['Worldwide Gross'] = df_list[9][1].iloc[3]
        except:
            movie_info['Worldwide Gross'] = None
        
        movie_info['Release Date'] = df_list[5][1].iloc[1][15:]
    
        for n in [12,13,14,15,16]:
            if df_list[n][0].iloc[0] == 'In Release:':      
                movie_info['Inrelease Time'] = df_list[n][1].iloc[0]
                df_list[n+1].set_index(0,inplace=True)
                
                try:
                    movie_info['Actors'] = df_list[n+1][1].loc['Actors:']
                except:
                    movie_info['Actors'] = None
                
                try:
                    movie_info['Director'] = df_list[n+1][1].loc['Director:']
                except:
                    movie_info['Director'] = None   
                
                try:
                    movie_info['Producer'] = df_list[n+1][1].loc['Producer:']
                except:
                    movie_info['Producer'] = None   
                
                break
            else:
                n = n + 1
   
        movie_info['Distributor'] = df_list[5][0].iloc[1][14:]
        
        mv = pd.DataFrame(list(movie_info.items()))
        mv.columns = ['Items',name]
        mv.set_index('Items',inplace=True)
#         mv.to_csv(path_or_buf=f'./{name}Basic_info.csv')
        mv
        return mv
    except:
        ValueError(f'{name} is not in the TOP100 of year from to {begin_year} to {end_year}')

In [ ]:
#df_list[5] = Distributor, Genre,MPAA Rating,Release Date,Runtime,Production Budget
#df_list[9] = gross Domestic Foreign Worldwide
#df_list[10] = opening weekend, opending weekend of total gross
#df_list[11] = widest release of theaters
#df_list[12] = close date
#df_list[13] = inrelease time period
#df_list[14] = Director, Actors, Producer, Cinematographer, Composer

In [ ]:
def data_movies_table(begin_year=1980,end_year=2018,input_name=None):  
    if not input_name == None:
        name_list = input_name
    else:
        name_list = list(find_all(begin_year,end_year).keys())
    movie_table = data_movie(name_list[0],begin_year,end_year)
    for name in name_list[1:]:
        try:
            movie_info = data_movie(name,begin_year,end_year)
            movie_table[name]=movie_info[name]
        except:
            continue

#     mv.to_csv(path_or_buf=f'./{begin_year}-{end_year}some movies basic info.csv')
    return movie_table.T 

In [ ]:
def get_review(name, page_length = 2, num = 20):
    import requests
    from bs4 import BeautifulSoup
    url = 'https://www.rottentomatoes.com/m/'+name+'/reviews/'
    response = requests.get(url)
    results_page = BeautifulSoup(response.content, 'lxml')
    results = results_page.find_all('div', class_ = 'the_review')
    page = 1
    reviews = list()
    next_page_results = list()

    while page < page_length:
        for item in results_page.find_all('a', class_ = 'btn btn-xs btn-primary-rt'):
            if item.find('span', class_ = "glyphicon glyphicon-chevron-right"):
                url = 'https://www.rottentomatoes.com' + item.get('href')
                response = requests.get(url)
                results_page = BeautifulSoup(response.content, 'lxml')
                next_page_results = results_page.find_all('div', class_ = 'the_review')
                results.extend(next_page_results)
                page+=1
                for result in results:
                    reviews.append(result.get_text())

    return reviews[:num]

In [ ]:
def get_name_review(year, num = 30, page_length = 2, length = 20):
    movies = get_movie_name(None, year)
    for i in range(len(movies)):
        movies[i] = movies[i].lower().replace(' ', '_').replace(':', '').replace(',', '_').replace('.', '_').replace("'", '').replace('!', '_').replace('?', '').replace('-', '_').replace('(', '').replace(')', '').replace('_a_new_dragon_tattoo_story','').replace('the_equalizer_2','equalizer_2').replace('dr__seuss_the_grinch_2018','the_grinch').replace('disneys_','')
        movies[i] = movies[i].lower().replace('___', '_')
        movies[i] = movies[i].lower().replace('__', '_')
    review_list = list()
    name_list = get_movie_name(None, year)[:num]
    for movie in movies[:num]:
        review = get_review(movie, page_length, length)
        if not review or len(review)<5:
            try:
                review = get_review(movie+f'_{year}', page_length, length)
            except:
                print('Do not find the movie')
        
        review_list.append(' '.join(review))
    name_review = list(zip(name_list, review_list))
    return name_review

In [ ]:
#Find top 100 movies in a specific year
def movie_from_year(year):
        
    import requests 
    from bs4 import BeautifulSoup
    
    url = 'https://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
    response = requests.get(url)
    results = BeautifulSoup(response.content, 'lxml')
    results = results.find_all('a')        
    
    url_list=[]
    name_list = []
    for result in results:
        if result.get('href') != None:
            if 'id='in result.get('href') and '#' not in result.get_text():
                url_list.append('https://www.boxofficemojo.com'+result.get('href'))
                name_list.append(result.get_text())
    return url_list,name_list

#Find movie urls and names from one year to the other
def find_from_to(yearx,yeary):
    
    import pandas as pd
    
    list_all_url = []
    list_all_name = []
    
    for year in range(yearx,yeary+1):
        url,name = movie_from_year(year)
        list_all_url.append(url)
        list_all_name.append(name)
    
    return list_all_url,list_all_name

#Find a movie's data with a provided url
def movie_data(url):
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    import requests
    response=requests.get(url)
    result = BeautifulSoup(response.content,'lxml')
    name = ['Name','Domestic Total Gross','Distributor','Release Time','Genre','Runtime','MPAA Rating','Production Budget','Worldwide Gross']
    data_list = [item.get_text() for item in result.find_all('b')[1:9]]
    data_list.append(result.find_all('b')[13].get_text())
    if 'Domestic Lifetime Gross:' in data_list[2]:
        data_list = [item.get_text() for item in result.find_all('b')[1:3]]
        new_list = [item.get_text() for item in result.find_all('b')[4:10]]
        data_list.extend(new_list)
        data_list.append(result.find_all('b')[14].get_text())
    if '$' not in data_list[-1]:
        data_list[-1] = 'N/A'
    a = pd.DataFrame(data_list,name)
    return np.transpose(a)

#Find top 100 movies' data
def movie_data_group(year):
    
    import pandas as pd
    url_list,name_list = movie_from_year(year)
    list_ = []
    for url in url_list:
        try:
            a = movie_data(url)
        except:
            continue
        list_.append(a)
    return pd.concat(list_)

#This function is used to draw a pie graph for the feature distributions in top 100 movies.
def pie_graph(data,feature):
    
    import pandas as pd
    import numpy as np  
    import matplotlib.mlab as mlab  
    import matplotlib.pyplot as plt 
    new_data = data.groupby(feature).size().sort_values(ascending = False)
    label = list(new_data.index)
    x = list(new_data)
    abc = [(label[i],x[i]) for i in range(len(label))]
    ab = abc[:5]
    new_label = [item[0] for item in ab]
    new_x = [item[1] for item in ab]
    new_label.append('Others')
    total = [item[1] for item in abc]
    new_x.append(sum(total)-sum(new_x))
    fig = plt.figure()
    plt.axes(aspect=1)
    plt.pie(new_x,labels=new_label,autopct='%1.2f%%',shadow= True) 
    plt.title(feature.capitalize()+" Distribution")
    plt.show()

In [ ]:
# There are some details need to be adjusted
# get_movie_urls: the first item is wrong
# data_movies_table: something is wrong when handling with lots of items
